# CDSR BiViewer

Extract the abstracts as well as their PICO fields to see if we can train good PICO vectors.

In [1]:
from cochranenlp.readers import biviewer

biviewer = biviewer.BiViewer()

### Extract PI[C]O and Abstracts

Comparator seems to be contained within Intervention. It will be tough to separate them...

In [2]:
%%time

def gen_pio(biviwer):
    for i, study in enumerate(biviewer):
        c, p = study.cochrane, study.pubmed
        cc = c['CHARACTERISTICS']

        yield p['title'], \
              p['pmid'], \
              c['CDNO'], \
              cc['CHAR_PARTICIPANTS'], \
              cc['CHAR_INTERVENTIONS'], \
              cc['CHAR_OUTCOMES'], \
              cc['CHAR_METHODS'], \
              c['TYPE'], \
              p['abstract']
        
data = list(gen_pio(biviewer))

df = pd.DataFrame(zip(*data)).T
df.columns = ['title', 'pmid', 'cdno', 'population', 'intervention', 'outcome', 'method', 'type', 'abstract']
for column in df.columns:
    df[column] = df[column].map(lambda s: ''.join(x if ord(x) < 128 else '' for x in s) if s else '') # drop non-ascii characters

df.to_csv('pico_cdsr.csv', index=False)

df

CPU times: user 3min 16s, sys: 9.92 s, total: 3min 26s
Wall time: 3min 49s
